This notebook aims to find the best ordering of targets, from a random subset of permutations

new objective: get the predictions of all chains and average them out and measure the performance -- Does an ensemble of chains lead to a clear improvement over just one chain?

In [107]:
import numpy as np
import pandas as pd
import itertools
import random

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error as mse, brier_score_loss
from chaining import Chain
import os
from scipy.stats import mode

In [108]:
def generate_permutations_with_order(variables, pairs_or_groups, num_permutations, constrained_elements=None, random_state=None):
    random.seed(random_state)
    permutations_list = [list(variables)]  # Add the original order only once as a list of strings
    
    while len(permutations_list) < num_permutations + 1: #+1 because the origin order is counted too
        perm = list(random.sample(variables, len(variables)))
        valid = True
        
        for pair_or_group in pairs_or_groups:
            idxs = [perm.index(var) for var in pair_or_group]
            if sorted(idxs) != idxs:
                valid = False
                break
        
        if valid:
            if constrained_elements:
            # Check if all constrained elements are present in the first positions of the permutation
                if all(elem in perm[:len(constrained_elements)] for elem in constrained_elements):
                    permutations_list.append(perm)
        
    
    return permutations_list

In [109]:
# TRY 2
def generate_permutations_with_order_2(variables, pairs_or_groups, num_permutations, constrained_elements=None, random_state=None):
    random.seed(random_state)
    
    # Generate all permutations of the variables
    permutations = itertools.permutations(variables)
    
    # Filter permutations according to the specified pairs or groups
    ordered_permutations = []
    for perm in permutations:
        valid = True
        for pair_or_group in pairs_or_groups:
            idxs = [perm.index(var) for var in pair_or_group]
            if sorted(idxs) != idxs:
                valid = False
                break
        if valid:
            ordered_permutations.append(perm)
    
    # If constrained elements are specified, ensure they appear at the beginning of each permutation
    if constrained_elements:
        constrained_permutations = []
        for perm in ordered_permutations:
            # Check if all constrained elements are present in the first positions of the permutation
            if all(elem in perm[:len(constrained_elements)] for elem in constrained_elements):
                constrained_permutations.append(perm)
        ordered_permutations = constrained_permutations
    
    # Select random permutations from the filtered permutations
    random_permutations = random.sample(ordered_permutations, min(num_permutations, len(ordered_permutations)))
    
    # Convert permutations to list of strings
    permutations_list = [list(variables)]  # Add the original order only once as a list of strings
    for perm in random_permutations:
        permutations_list.append(list(perm))  # Convert each permutation to a list of strings
    
    return permutations_list

In [110]:
def missingness_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [111]:
possible_paths = [
    'C:/Users/lenne/OneDrive/Documenten/Master of Statistics and Data Science/2023-2024/Master thesis/Thesis_Sofia_Lennert/new_data',
    'C:/Users/anaso/Desktop/SOFIA MENDES/KU Leuven/Master Thesis/Thesis_Sofia_Lennert/new_data'
]

# Define file names
file = 'merged_data.csv'

# Find full paths to the CSV files
path = next((f'{path}/{file}' for path in possible_paths if os.path.exists(f'{path}/{file}')), None)

# Resulting DataFrame will have aggregated data from all four datasets based on the specific_column
pd.set_option('display.max_columns', None)

data = pd.read_csv(path)

def bin_column(value):
    if value in [0, 1, 2, 3]:
        return str(value)
    else:
        return '4+'
data['NRELAP'] = data['NRELAP'].apply(bin_column)

data

,USUBJID,AGE,SEX,RACE,CONTINENT,CESEV,CECONTRT,TOTRELAP,MHCONTRT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,SMSTDY,NRELAP,NHPT-before,NHPT-2y,NHPT-after_2y,PASAT_2s-before,PASAT_2s-2y,PASAT_2s-after_2y,PASAT_3s-before,PASAT_3s-2y,PASAT_3s-after_2y,SDMT-before,SDMT-2y,T25FW-before,T25FW-2y,T25FW-after_2y,T-before,T-after,P-before,P-after,N-before,N-after,SLEC_before,SLEC_after,SES_after,SES_before,VAA,BDI-before,BDI-after,EDSS-before,EDSS-2y,EDSS-after_2y,KFSS1-Sensory-2y,KFSS1-Sensory-after_2y,KFSS1-Sensory-before,KFSS1-Brain-2y,KFSS1-Brain-after_2y,KFSS1-Brain-before,KFSS1-Bowel-2y,KFSS1-Bowel-after_2y,KFSS1-Bowel-before,KFSS1-Pyramidal-2y,KFSS1-Pyramidal-after_2y,KFSS1-Pyramidal-before,KFSS1-Cerebral-2y,KFSS1-Cerebral-after_2y,KFSS1-Cerebral-before,KFSS1-Visual-2y,KFSS1-Visual-after_2y,KFSS1-Visual-before,KFSS1-Cerebellar-2y,KFSS1-Cerebellar-after_2y,KFSS1-Cerebellar-before,KFSS_M-2y,KFSS_M-after_2y,KFSS_M-before,KFSS_P-2y,KFSS_P-after_2y,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,M_R36-SF12-after,P_R36-SF12-after,R36-SF12-after_Ind
0,MSOAC/0014,46.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSOAC/0016,NaN,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,Y,SPMS,1,1,0,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.55,6.60,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MSOAC/0019,44.0,M,NON-WHITE,NaN,NaN,NaN,NaN,NaN,PPMS,1,1,0,0,NaN,0.0,23.65,21.30,20.15,34.5,35.5,43.0,43.5,51.0,53.0,NaN,NaN,6.30,6.15,5.85,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.75,3.50,3.0,0.333333,0.166667,0.500000,0.2,0.0,0.2,0.000000,0.166667,0.083333,0.333333,0.5,0.416667,0.0,0.0,0.0,0.333333,0.0,0.333333,0.0,0.2,0.0,0.185185,0.185185,0.240741,0.166667,0.083333,0.208333,0.828571,0.772152,1.0,0.857143,0.721519,1.0
3,MSOAC/0024,60.0,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,NaN,SPMS,1,1,1,1,NaN,0.0,34.45,37.50,NaN,55.0,54.0,NaN,60.0,60.0,NaN,NaN,NaN,4.50,5.25,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.031746,0.023810,4.00,3.75,NaN,0.333333,NaN,0.333333,0.0,NaN,0.1,0.583333,NaN,0.666667,0.166667,NaN,0.250000,0.0,NaN,0.0,0.000000,NaN,0.083333,0.2,NaN,0.5,0.129630,NaN,0.240741,0.291667,NaN,0.375000,0.885714,0.569620,1.0,0.857143,0.716216,1.0
4,MSOAC/0030,28.0,F,WHITE,EUROPE,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,16.55,17.90,NaN,NaN,NaN,NaN,58.0,60.0,NaN,63.5,69.0,4.85,4.70,NaN,0.0,0.0,0.0,0.0,0.0,0.0,26.0,24.0,1.25,1.25,NaN,0.063492,0.039683,2.00,1.50,NaN,0.166667,NaN,0.166667,0.2,NaN,0.2,0.166667,NaN,0.166667,0.166667,NaN,0.333333,0.0,NaN,0.2,0.166667,NaN,0.083333,0.0,NaN,0.1,0.111111,NaN,0.203704,0.166667,NaN,0.125000,0.933333,0.846154,0.0,0.833333,0.730769,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,MSOAC/9986,46.0,M,WHITE,OCEANIA,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,19.35,18.95,NaN,NaN,NaN,NaN,58.0,60.0,NaN,51.0,60.0,3.90,3.80,NaN,0.0,0.0,0.0,0.0,0.0,0.0,36.0,35.0,1.25,1.25,NaN,0.047619,0.063492,2.75,2.50,NaN,0.333333,NaN,0.166667,0.0,NaN,0.0,0.333333,NaN,0.250000,0.166667,NaN,0.333333,0.0,NaN,0.0,0.000000,NaN,0.000000,0.0,NaN,0.2,0.111111,NaN,0.148148,0.166667,NaN,0.125000,0.833333,0.730769,0.0,0.800000,0.750000,0.0
2461,MSOAC/9987,18.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [112]:
variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 'M_R36-SF12-after', 
             'SES_after', 'SLEC_after', 'KFSS_M-after_2y', 'KFSS_P-after_2y', 'EDSS-after_2y', 'NRELAP', 'CESEV']


columns_to_keep = ['AGE', 'SEX', 'RACE', 'CONTINENT', 'MHDIAGN', 'CARDIO', 'URINARY', 'MUSCKELET', 'FATIGUE', 
                    'NHPT-before', 'PASAT_2s-before', 'PASAT_3s-before', 'SDMT-before', 'T25FW-before', 'SLEC_before','SES_before',
                    'BDI-before', 'EDSS-before', 'KFSS_M-before', 'KFSS_P-before', 'M_R36-SF12-before',
                	'P_R36-SF12-before', 'R36-SF12-before_Ind', 'T-before','P-before','N-before']
# still need to change in OE dataframe the SLEC and SES so name is consistent with the others

features = data[columns_to_keep]
features

,AGE,SEX,RACE,CONTINENT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before
0,46.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,M,WHITE,NORTH AMERICA,SPMS,1,1,0,1,NaN,NaN,NaN,NaN,8.55,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,44.0,M,NON-WHITE,NaN,PPMS,1,1,0,0,23.65,34.5,43.5,NaN,6.30,NaN,NaN,NaN,3.75,0.240741,0.208333,0.828571,0.772152,1.0,0.0,0.0,NaN
3,60.0,M,WHITE,NORTH AMERICA,SPMS,1,1,1,1,34.45,55.0,60.0,NaN,4.50,NaN,NaN,0.031746,4.00,0.240741,0.375000,0.885714,0.569620,1.0,0.0,0.0,1.0
4,28.0,F,WHITE,EUROPE,RRMS,1,1,0,1,16.55,NaN,58.0,63.5,4.85,26.0,1.25,0.063492,2.00,0.203704,0.125000,0.933333,0.846154,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,46.0,M,WHITE,OCEANIA,RRMS,1,1,0,1,19.35,NaN,58.0,51.0,3.90,36.0,1.25,0.047619,2.75,0.148148,0.125000,0.833333,0.730769,0.0,0.0,0.0,0.0
2461,18.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2462,38.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2463,40.0,F,WHITE,NaN,PPMS,0,1,0,1,23.80,21.5,31.5,NaN,6.15,NaN,NaN,NaN,4.50,0.481481,0.166667,0.728571,0.658228,1.0,0.0,0.0,NaN


In [113]:
object_columns = features.select_dtypes(include=['object'])
features = pd.get_dummies(features, columns=object_columns.columns, dtype=int)
features.head()

,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS
0,46.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,1,0
1,NaN,1,1,0,1,NaN,NaN,NaN,NaN,8.55,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,1,0,1,0,0,0,1,0,0,0,0,1
2,44.0,1,1,0,0,23.65,34.5,43.5,NaN,6.30,NaN,NaN,NaN,3.75,0.240741,0.208333,0.828571,0.772152,1.0,0.0,0.0,NaN,0,1,1,0,0,0,0,0,0,0,1,0,0
3,60.0,1,1,1,1,34.45,55.0,60.0,NaN,4.50,NaN,NaN,0.031746,4.00,0.240741,0.375000,0.885714,0.569620,1.0,0.0,0.0,1.0,0,1,0,1,0,0,0,1,0,0,0,0,1
4,28.0,1,1,0,1,16.55,NaN,58.0,63.5,4.85,26.0,1.25,0.063492,2.00,0.203704,0.125000,0.933333,0.846154,0.0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,0,0,1,0


In [114]:
#variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 
             #'M_R36-SF12-after', 'SES_after', 'EDSS-after_2y', 'NRELAP', 'CESEV']

pairs_or_groups = [['KFSS_M-2y', 'EDSS-2y'], ['KFSS_P-2y', 'EDSS-2y'], ['KFSS_M-after_2y', 'EDSS-after_2y'], ['KFSS_P-after_2y', 'EDSS-after_2y']]  # Specify the pairs or groups
order_constraint = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y']
num_permutations = 9  # Specify how many random permutations you want
random_state = 42

random_permutations = generate_permutations_with_order(variables, pairs_or_groups, num_permutations, order_constraint, random_state)

# Print the original order followed by all the random permutations
for idx, perm in enumerate(random_permutations, start=0):
    print(f"Permutation {idx}: {', '.join(perm)}")

Permutation 0: KFSS_M-2y, KFSS_P-2y, EDSS-2y, T25FW-2y, NHPT-2y, P_R36-SF12-after, M_R36-SF12-after, SES_after, SLEC_after, KFSS_M-after_2y, KFSS_P-after_2y, EDSS-after_2y, NRELAP, CESEV
Permutation 1: T25FW-2y, KFSS_M-2y, NHPT-2y, KFSS_P-2y, EDSS-2y, NRELAP, KFSS_M-after_2y, CESEV, SLEC_after, M_R36-SF12-after, P_R36-SF12-after, KFSS_P-after_2y, EDSS-after_2y, SES_after
Permutation 2: NHPT-2y, KFSS_P-2y, KFSS_M-2y, T25FW-2y, EDSS-2y, M_R36-SF12-after, SES_after, CESEV, SLEC_after, KFSS_P-after_2y, P_R36-SF12-after, KFSS_M-after_2y, NRELAP, EDSS-after_2y
Permutation 3: NHPT-2y, KFSS_P-2y, KFSS_M-2y, T25FW-2y, EDSS-2y, KFSS_P-after_2y, CESEV, M_R36-SF12-after, NRELAP, P_R36-SF12-after, SES_after, SLEC_after, KFSS_M-after_2y, EDSS-after_2y
Permutation 4: KFSS_P-2y, KFSS_M-2y, NHPT-2y, T25FW-2y, EDSS-2y, CESEV, KFSS_M-after_2y, P_R36-SF12-after, KFSS_P-after_2y, SES_after, SLEC_after, EDSS-after_2y, M_R36-SF12-after, NRELAP
Permutation 5: T25FW-2y, KFSS_M-2y, KFSS_P-2y, EDSS-2y, NHPT-2y, 

In [115]:
ordered_targets = random_permutations[0]

In [116]:
# Set random state for reproducibility
random_state = 42
N_FOLDS = 5

In [117]:
# Generate CV folds
cv=missingness_stratified_cv(features, N_FOLDS, random_state)
cv = cv.to_frame(name="CV Fold")

Is it a problem that not all folds have the exact same number?

---

In [118]:
# Define a function to reorder columns of dataframes
def reorder_columns(dataframes):
    # Get the column order from the first dataframe
    column_order = dataframes[0].columns
    # Reorder columns for each dataframe in the list
    reordered_dataframes = [df[column_order] for df in dataframes]
    return reordered_dataframes

def average_dataframes(dataframes):
    # Concatenate dataframes within the list
    concatenated_df = pd.concat(dataframes)
    # Group by index and calculate the mode for object columns and mean for other types
    averaged_df = concatenated_df.groupby(concatenated_df.index).agg(lambda x: x.mode()[0] if x.dtype == 'O' else x.mean())
    return averaged_df

---

## Chain with *true* values propagated

In [119]:
y_pred_chains = []
y_test_list = [[] for _ in range(N_FOLDS)]  # Initialize y_test_list with empty lists for each fold index
y_pred_prob_list_chain = []
yi_test_dummies_list = [[] for _ in range(N_FOLDS)]


# Iterate over each chain ordering
for ordered_targets_chain in random_permutations:
    y_pred_list = []  # List to store predictions for this chain
    y_pred_prob_list = []
    
    features_cv = pd.merge(features, pd.DataFrame(cv), left_index=True, right_index=True)
    targets_cv = pd.merge(data[ordered_targets_chain], pd.DataFrame(cv), left_index=True, right_index=True)

    # Fit and predict for each fold for this chain
    for i in range(0, N_FOLDS): 
        Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
        Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
        yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
        yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
        y_test_list_chain = []  # List to store y_test for this fold

        # One hot encode categorical targets of test set to be able to compute brier score
        subset_yi_test = yi_test.select_dtypes(include=['object'])
        yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)

        chain = Chain(
            model_reg=RandomForestRegressor(random_state=random_state),
            model_clf=RandomForestClassifier(random_state=random_state),
            propagate="true",
        )
        chain.fit(Xi_train, yi_train, target_types=None)
        y_pred = chain.predict(Xi_test)
        y_pred_prob = chain.predict_proba(Xi_test)
        y_pred_list.append(pd.DataFrame(y_pred, columns=yi_test.columns, index=yi_test.index))
        y_pred_prob_list.append(y_pred_prob)
    
        # Append yi_test to the corresponding fold index in y_test_list
        y_test_list[i].append(yi_test)  # Append yi_test for fold i
        yi_test_dummies_list[i].append(yi_test_dummies)

    y_pred_chains.append(y_pred_list)
    y_pred_prob_list_chain.append(y_pred_prob_list)
    print("Permutation done")

Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done


In [120]:
yi_test_dummies_list[3][3]

,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+
4,0,0,0,1,0,0,0,0
12,0,0,1,0,0,1,0,0
14,0,0,1,0,1,0,0,0
19,0,1,0,0,1,0,0,0
24,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2436,0,0,0,1,0,0,0,0
2439,1,0,0,0,1,0,0,0
2445,0,0,0,1,0,0,0,0
2462,1,0,0,0,0,1,0,0


In [121]:
y_test_list

[[      KFSS_M-2y  KFSS_P-2y  EDSS-2y  T25FW-2y  NHPT-2y  P_R36-SF12-after  \
  0           NaN        NaN      NaN       NaN      NaN               NaN   
  3      0.129630   0.291667     3.75      5.25    37.50          0.716216   
  8           NaN        NaN     6.50     22.90    31.55          0.500000   
  13          NaN        NaN     8.00       NaN    37.90          0.468354   
  20     0.259259   0.166667     4.00      6.50    30.55          0.582278   
  ...         ...        ...      ...       ...      ...               ...   
  2449   0.333333   0.000000     4.00     12.25    27.25          0.346154   
  2454   0.240741   0.541667     6.50     11.55    28.50          0.443038   
  2457   0.259259   0.000000     3.50      5.30    23.00          0.807692   
  2458        NaN        NaN     4.00      7.95    28.50          0.683544   
  2460   0.111111   0.166667     2.50      3.80    18.95          0.750000   
  
        M_R36-SF12-after  SES_after  SLEC_after  KFSS_M-after

In [122]:
yi_test_dummies_list[2][0]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
2,1,0,0,0,0,0,0,0
5,0,0,1,0,0,1,0,0
15,1,0,0,0,0,0,0,0
23,1,0,0,0,0,0,0,0
31,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...
2441,1,0,0,0,0,0,0,0
2446,0,0,1,0,0,0,1,0
2450,1,0,0,0,0,0,0,0
2452,1,0,0,0,0,0,0,0


In [123]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs_chain = []

for y_pred_prob_list in y_pred_prob_list_chain:
    concatenated_dfs_fold = []
    # Iterate over each pair of arrays
    for j, fold in enumerate(y_pred_prob_list):
        dfs = []
        len_array = 0
        
        for i, array in enumerate(fold):
            # Convert array to DataFrame
            col = yi_test_dummies_list[j][0].columns[len_array:len_array+len(array[0])]
            df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j][0].index)
            dfs.append(df)
            len_array += len(array[0])
        
        # Concatenate DataFrames
        concatenated_df = pd.concat(dfs, axis=1)
        concatenated_dfs_fold.append(concatenated_df)

    concatenated_dfs_chain.append(concatenated_dfs_fold)
    # Now you should have a list of concatenated DataFrames

In [124]:
concatenated_dfs_chain[0][3]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
4,0.590000,0.300000,0.040000,0.020000,0.050000,0.210000,0.700000,0.090000
12,0.601269,0.198408,0.130323,0.030000,0.040000,0.312083,0.617917,0.070000
14,0.630000,0.250000,0.050000,0.040000,0.030000,0.070000,0.650000,0.280000
19,0.575726,0.142917,0.210107,0.031250,0.040000,0.242500,0.667500,0.090000
24,0.400931,0.235709,0.132717,0.081943,0.148700,0.253751,0.609397,0.136852
...,...,...,...,...,...,...,...,...
2436,0.800000,0.100000,0.070000,0.030000,0.000000,0.250000,0.440000,0.310000
2439,0.300000,0.240000,0.196833,0.163167,0.100000,0.295976,0.594024,0.110000
2445,0.560000,0.200000,0.180000,0.040000,0.020000,0.550000,0.400000,0.050000
2462,0.384804,0.214254,0.182305,0.070000,0.148637,0.298573,0.607566,0.093861


In [125]:
transposed_list_cat = list(zip(*concatenated_dfs_chain))
transposed_list_cat

[(      NRELAP_0.0  NRELAP_1.0  NRELAP_2.0  NRELAP_3.0  NRELAP_4+  CESEV_MILD  \
  0       0.394247    0.375554    0.132055    0.068145       0.03    0.322599   
  3       0.740000    0.180000    0.010000    0.030000       0.04    0.190000   
  8       0.710000    0.180000    0.070000    0.020000       0.02    0.260000   
  13      0.570000    0.260000    0.110000    0.050000       0.01    0.216667   
  20      0.520000    0.150000    0.240000    0.020000       0.07    0.070000   
  ...          ...         ...         ...         ...        ...         ...   
  2449    0.560000    0.290000    0.110000    0.020000       0.02    0.240000   
  2454    0.890000    0.030000    0.070000    0.010000       0.00    0.192000   
  2457    0.410000    0.380000    0.130000    0.040000       0.04    0.250000   
  2458    0.800000    0.120000    0.050000    0.030000       0.00    0.200000   
  2460    0.610000    0.250000    0.070000    0.060000       0.01    0.230000   
  
        CESEV_MODERATE  C

In [126]:
# Each element of transposed_list is a tuple containing dataframes from the same position in each inner list
# Now, create a list of lists from these tuples
reorganized_list_cat = [list(df_tuple) for df_tuple in transposed_list_cat]
reorganized_list_cat

[[      NRELAP_0.0  NRELAP_1.0  NRELAP_2.0  NRELAP_3.0  NRELAP_4+  CESEV_MILD  \
  0       0.394247    0.375554    0.132055    0.068145       0.03    0.322599   
  3       0.740000    0.180000    0.010000    0.030000       0.04    0.190000   
  8       0.710000    0.180000    0.070000    0.020000       0.02    0.260000   
  13      0.570000    0.260000    0.110000    0.050000       0.01    0.216667   
  20      0.520000    0.150000    0.240000    0.020000       0.07    0.070000   
  ...          ...         ...         ...         ...        ...         ...   
  2449    0.560000    0.290000    0.110000    0.020000       0.02    0.240000   
  2454    0.890000    0.030000    0.070000    0.010000       0.00    0.192000   
  2457    0.410000    0.380000    0.130000    0.040000       0.04    0.250000   
  2458    0.800000    0.120000    0.050000    0.030000       0.00    0.200000   
  2460    0.610000    0.250000    0.070000    0.060000       0.01    0.230000   
  
        CESEV_MODERATE  C

In [127]:
# Apply the function to each list in reorganized_list
reordered_reorganized_list_cat = [reorder_columns(dataframes) for dataframes in reorganized_list_cat]

In [128]:
# Apply the function to each list in reordered_reorganized_list
averaged_dataframes_list_cat = [average_dataframes(dataframes) for dataframes in reordered_reorganized_list_cat]

In [129]:
averaged_dataframes_list_cat[4]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
6,0.345068,0.481665,0.159267,0.358055,0.201117,0.185813,0.209014,0.060000
9,0.364174,0.442894,0.173031,0.309535,0.209605,0.234274,0.196541,0.069944
10,0.243000,0.672000,0.077000,0.414000,0.229000,0.073000,0.246000,0.046000
16,0.413699,0.346455,0.219676,0.390768,0.156391,0.212076,0.180670,0.080264
17,0.294000,0.627000,0.078000,0.529000,0.114000,0.068000,0.248000,0.042000
...,...,...,...,...,...,...,...,...
2438,0.221000,0.697000,0.076000,0.368000,0.192000,0.128000,0.287000,0.031000
2443,0.358000,0.565000,0.068000,0.578000,0.071000,0.087000,0.239000,0.034000
2451,0.391000,0.445000,0.121000,0.485417,0.106000,0.151583,0.222000,0.078000
2453,0.290000,0.586000,0.116000,0.480000,0.113000,0.120000,0.250000,0.045000


In [130]:
transposed_list_all = list(zip(*y_pred_chains))

# Each element of transposed_list is a tuple containing dataframes from the same position in each inner list
# Now, create a list of lists from these tuples
reorganized_list_all = [list(df_tuple) for df_tuple in transposed_list_all]

# Apply the function to each list in reorganized_list
reordered_reorganized_list_all = [reorder_columns(dataframes) for dataframes in reorganized_list_all]

# Apply the function to each list in reordered_reorganized_list
averaged_dataframes_list_all = [average_dataframes(dataframes) for dataframes in reordered_reorganized_list_all]

In [131]:
transposed_test_list = list(zip(*y_test_list))

# Each element of transposed_list is a tuple containing dataframes from the same position in each inner list
# Now, create a list of lists from these tuples
reorganized_test_list = [list(df_tuple) for df_tuple in transposed_test_list]

In [132]:
reorganized_test_list[0]

[      KFSS_M-2y  KFSS_P-2y  EDSS-2y  T25FW-2y  NHPT-2y  P_R36-SF12-after  \
 0           NaN        NaN      NaN       NaN      NaN               NaN   
 3      0.129630   0.291667     3.75      5.25    37.50          0.716216   
 8           NaN        NaN     6.50     22.90    31.55          0.500000   
 13          NaN        NaN     8.00       NaN    37.90          0.468354   
 20     0.259259   0.166667     4.00      6.50    30.55          0.582278   
 ...         ...        ...      ...       ...      ...               ...   
 2449   0.333333   0.000000     4.00     12.25    27.25          0.346154   
 2454   0.240741   0.541667     6.50     11.55    28.50          0.443038   
 2457   0.259259   0.000000     3.50      5.30    23.00          0.807692   
 2458        NaN        NaN     4.00      7.95    28.50          0.683544   
 2460   0.111111   0.166667     2.50      3.80    18.95          0.750000   
 
       M_R36-SF12-after  SES_after  SLEC_after  KFSS_M-after_2y  \
 0     

In [133]:
y_pred_list = averaged_dataframes_list_all.copy()
y_test_list = reorganized_test_list[0]

y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [134]:
def normalized_mean_squared_error(true, pred):
    num = mse(true, pred)
    mean_value = np.mean(true)
    mean = np.full_like(true, mean_value)
    den = mse(true, mean)
    nmse_loss = num/den
    #rrmse_loss = np.sqrt(squared_error)
    return nmse_loss

In [135]:
# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            
            score = normalized_mean_squared_error(y_test, y_pred)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - true values):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - true values):
KFSS_M-2y: 0.19 (± 0.02)
KFSS_P-2y: 0.25 (± 0.03)
EDSS-2y: 0.12 (± 0.02)
T25FW-2y: 0.29 (± 0.08)
NHPT-2y: 0.41 (± 0.18)
P_R36-SF12-after: 0.31 (± 0.05)
M_R36-SF12-after: 0.43 (± 0.02)
SES_after: 0.31 (± 0.05)
SLEC_after: 0.36 (± 0.03)
KFSS_M-after_2y: 0.35 (± 0.03)
KFSS_P-after_2y: 0.48 (± 0.06)
EDSS-after_2y: 0.24 (± 0.05)


In [136]:
yi_test_dummies_avg = []

for yi_test_dummies_fold in yi_test_dummies_list:
    # Calculate the percentage of 1s in each column
    yi_test_dummies_avg_fold=[]
    for yi_test_dummies_chain in yi_test_dummies_fold:

        percentages = yi_test_dummies_chain.sum() / len(yi_test_dummies_chain)

        yi_test_dummies_avg_chain = pd.DataFrame(0, index=yi_test_dummies_chain.index, columns=yi_test_dummies_chain.columns)

        # Replace values in each column with the corresponding percentage
        for col in yi_test_dummies_avg_chain.columns:
            yi_test_dummies_avg_chain[col] = yi_test_dummies_chain[col].apply(lambda x: percentages[col])

        yi_test_dummies_avg_fold.append(yi_test_dummies_avg_chain)
    yi_test_dummies_avg.append(yi_test_dummies_avg_fold)

In [137]:
transposed_dummy_avg_list = list(zip(*yi_test_dummies_avg))

# Each element of transposed_list is a tuple containing dataframes from the same position in each inner list
# Now, create a list of lists from these tuples
reorganized_dummy_avg_list = [list(df_tuple) for df_tuple in transposed_dummy_avg_list]

In [138]:
reorganized_dummy_avg_list[0][0]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
0,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
3,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
8,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
13,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
20,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
...,...,...,...,...,...,...,...,...
2449,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
2454,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
2457,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
2458,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058


In [139]:
transposed_dummy_list = list(zip(*yi_test_dummies_list))

# Each element of transposed_list is a tuple containing dataframes from the same position in each inner list
# Now, create a list of lists from these tuples
reorganized_dummy_list = [list(df_tuple) for df_tuple in transposed_dummy_list]

In [140]:
reorganized_dummy_list[3][0]

,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+
0,0,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0,0
8,0,0,0,1,0,0,0,0
13,0,1,0,1,0,0,0,0
20,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
2449,0,0,0,1,0,0,0,0
2454,0,0,0,1,0,0,0,0
2457,0,0,0,1,0,0,0,0
2458,0,0,0,1,0,0,0,0


In [141]:
reorganized_dummy_avg_list_first=reorganized_dummy_avg_list[0]
reorganized_dummy_list_first=reorganized_dummy_list[0]

In [142]:
reorganized_dummy_list_first[3]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
4,1,0,0,0,0,0,0,0
12,0,0,1,0,0,0,0,1
14,0,1,0,0,0,0,0,1
19,0,1,0,0,0,0,1,0
24,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
2436,1,0,0,0,0,0,0,0
2439,0,1,0,0,0,1,0,0
2445,1,0,0,0,0,0,0,0
2462,0,0,1,0,0,1,0,0


In [143]:
reorganized_dummy_avg_list_first[1]['NRELAP_0.0']

1       0.646465
7       0.646465
11      0.646465
21      0.646465
30      0.646465
          ...   
2437    0.646465
2447    0.646465
2455    0.646465
2459    0.646465
2464    0.646465
Name: NRELAP_0.0, Length: 495, dtype: float64

In [144]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = reorganized_dummy_list_first[0].columns

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = reorganized_dummy_list_first[fold_index][level_name] 
        y_prob = averaged_dataframes_list_cat[fold_index][level_name] 
        y_prob_avg = reorganized_dummy_avg_list_first[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((level_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each level:")
for level_name, avg_score, std_score in scores_with_std:
    print(f"{level_name}: {avg_score:.2f} (± {std_score:.2f})")

Normalized Brier scores for each level:
NRELAP_0.0: 1.40 (± 0.09)
NRELAP_1.0: 1.54 (± 0.15)
NRELAP_2.0: 1.13 (± 0.07)
NRELAP_3.0: 6.79 (± 1.49)
NRELAP_4+: 1.71 (± 0.19)
CESEV_MILD: 1.00 (± 0.02)
CESEV_MODERATE: 0.99 (± 0.01)
CESEV_SEVERE: 0.98 (± 0.01)


In [145]:
# Initialize dictionary to store column sums
column_sums = {}

# Iterate over each dataframe in reorganized_dummy_list_first
for df in reorganized_dummy_list_first:
    # Iterate over each column in the dataframe
    for column in df.columns:
        # Sum occurrences of 1s in the column and update column_sums
        column_sum = df[column].sum()
        column_sums[column] = column_sums.get(column, 0) + column_sum

# Create a dataframe from the column sums
total_counts_df = pd.DataFrame(list(column_sums.items()), columns=['Name', 'Total'])
total_counts_df

,Name,Total
0,NRELAP_0.0,1612
1,NRELAP_1.0,485
2,NRELAP_2.0,211
3,NRELAP_3.0,82
4,NRELAP_4+,75
5,CESEV_MILD,238
6,CESEV_MODERATE,534
7,CESEV_SEVERE,168


In [146]:
# Calculate the weighted sum for NRELAP
nrelap_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("NRELAP"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        nrelap_weighted_sum += avg_score * count
        total_count += count
nrelap_weighted_avg = nrelap_weighted_sum / total_count

# Calculate the weighted sum for CESEV
cesev_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("CESEV"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        cesev_weighted_sum += avg_score * count
        total_count += count
cesev_weighted_avg = cesev_weighted_sum / total_count

# Print the results
print(f"Weighted avg for NRELAP: {nrelap_weighted_avg:.2f}")
print(f"Weighted avg for CESEV: {cesev_weighted_avg:.2f}")

cat_normalized_brier= [nrelap_weighted_avg, cesev_weighted_avg]

Weighted avg for NRELAP: 1.59
Weighted avg for CESEV: 0.99


In [147]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = reorganized_dummy_list_first[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = reorganized_dummy_list_first[fold_index][level_name] 
        y_prob = averaged_dataframes_list_cat[fold_index][level_name] 
        y_prob_avg = reorganized_dummy_avg_list_first[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

#cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    #cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 2.51 (± 2.25)
CESEV: 0.99 (± 0.02)


In [148]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.19253563, 0.25308166, 0.11728709, 0.29047024, 0.40654838,
       0.30702935, 0.43213423, 0.30755793, 0.3642841 , 0.34561239,
       0.47937319, 0.24317007, 1.59061662, 0.9909569 ])

In [149]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 0.4514755570992551
